In [1]:
import gradio as gr
import time
from tqdm import tqdm
import re
import modules.callbacks
import PIL

In [2]:
def fake_gpt(text,duration):
    splitted_text = text.split(' ')
    
    for i in tqdm(range(len(splitted_text))):
        time.sleep(float(duration))
        joined_text = ""
        for j in range(i+1):
            joined_text += splitted_text[j] + " "
        yield joined_text, joined_text.title(), joined_text.upper(), joined_text.lower()

In [3]:
def count_sentences(text):
    #From text, count how many . ! ? there are
    #If . ! ? are followed by a space,double mark or a new line, then it is a sentence
    #If . ! ? in the end of the text, then it is a sentence
    return len(re.findall(r'[\.\!\?][\s\”\'\n]',text)) + len(re.findall(r'[\.\!\?]$',text))


In [4]:
import gc
import torch

def clear_torch_cache():
    gc.collect()
    torch.cuda.empty_cache()

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path_or_name = "/home/karuniaperjuangan/AI-Project/text-generation-webui/models/gpt2-small-indonesian-story"

model = AutoModelForCausalLM.from_pretrained(model_path_or_name, torch_dtype=torch.float16).cuda()
tokenizer = AutoTokenizer.from_pretrained(model_path_or_name,truncation_side="left")

In [6]:
def stream_generate_text(init_text,max_sentences=0,max_tokens=512,hidden_texts=[]):
    """
    init_text: initial text that will be used to generate text
    max_sentences: maximum number of sentences to generate
    max_tokens: maximum number of tokens to generate
    hidden_sentence: list of string that will be hidden from the generated text (example: initial text and unfinished sentences)
    """
    sentence = init_text
    inputs_ids = tokenizer(sentence, return_tensors="pt").input_ids.cuda()
    continue_generate = True
    while continue_generate:
        inputs_ids = tokenizer(sentence, return_tensors="pt").input_ids.cuda()
        generate_params = {
        "max_new_tokens": 8,
        "eos_token_id": tokenizer.eos_token_id,
        "pad_token_id": tokenizer.eos_token_id,
        "top_k": 50,
        "top_p": 0.95,
        "temperature": 0.9,
        "repetition_penalty": 1.2,
        "do_sample": True,
        }
        generate_params["input_ids"] = inputs_ids
        outputs = model.generate(**generate_params)
        sentence = tokenizer.decode(outputs[0])
        inputs_ids = outputs[0]
        #print(count_sentences(sentence)) 
        outputted_sentences = sentence
        for hidden in hidden_texts:
            outputted_sentences = outputted_sentences.replace(hidden,"")
        yield outputted_sentences
            
        if count_sentences(sentence) >= max_sentences or ( len(inputs_ids)-len(tokenizer.encode(init_text)) ) > max_tokens or tokenizer.eos_token_id in inputs_ids:
            continue_generate = False
    #Clear text after last ,.",!",?",.,?,!
    deleted_text = re.split(r"[\.\!\?]",sentence)[-1]

    outputted_sentences = sentence.replace(deleted_text,"")
    for hidden in hidden_texts:
        outputted_sentences = outputted_sentences.replace(hidden,"")
    yield outputted_sentences

In [12]:
story_sequence = []

def generate_story_and_choices(beginning_text, selected_choice="", num_choices=3, story_beginning=True):
    """
    beginning_text: text that will be used to generate story (if in beginning is the story, else the choice of previous story)
    num_choices: number of choices to generate
    story_beginning: True if the beginning_text is the story, else False
    """
    global story_sequence
    if story_beginning:
        story_sequence = []
        story = beginning_text
    else:
        story = ""
        for item in story_sequence:
            story += item
    pregenerated_length = len(story)
    
    story += selected_choice

    
    choices = ["..." for i in range(num_choices)]

    for text in stream_generate_text(story,max_sentences=5,max_tokens=512):
        story = text
        # Di awal, juga tampilkan awalan cerita yang dimasukkan pengguna
        yield story[pregenerated_length:] if not story_beginning else story, *choices
    
    #clean story from html tag
    story = re.sub(r"<[^>]*>","",story)
    
    story_sequence.append(story[pregenerated_length:] if not story_beginning else story)
    #Generate choices
    max_sentences = 2+count_sentences(story)
    for i in range(num_choices):
        choices[i] = ""
        for text in stream_generate_text(story,max_sentences=max_sentences,max_tokens=512):
            choices[i] = text[len(story):]
            yield story[pregenerated_length:], *choices
        #clean choices from html tag
        choices[i] = re.sub(r"<[^>]*>","",choices[i])
        
def select_choice(choice):
    global story_sequence
    full_story = ""
    for item in story_sequence:
        full_story += item
    for iter in generate_story_and_choices("",selected_choice=choice,story_beginning=False):
        yield iter

In [8]:
from PIL import Image
blank_image = Image.new('RGB', (768, 768), (255,255,255))

In [15]:
with gr.Blocks(css='./style.css') as blocks:
    with gr.Column():
        with gr.Row():
            text = gr.Textbox(label="Text", value="Pada zaman dahulu,")
            button = gr.Button(value="Generate")
        with gr.Column():
            with gr.Row():
                output = gr.Markdown(value="This is a test")
                #image = gr.Gallery(value=[blank_image])
            with gr.Row(elem_classes="choicerow"):
                button_1 = gr.Button(value="Select", elem_classes="choicebutton")
                choice_1 = gr.Markdown(value="This is a choice 1",elem_classes="choice-label")
            with gr.Row(elem_classes="choicerow"):
                button_2 = gr.Button(value="Select", elem_classes="choicebutton")
                choice_2 = gr.Markdown(value="This is a choice 1",elem_classes="choice-label")
            with gr.Row(elem_classes="choicerow"):
                button_3 = gr.Button(value="Select", elem_classes="choicebutton")
                choice_3 = gr.Markdown(value="This is a choice 1",elem_classes="choice-label")
                

        with gr.Row():
            pass
    button.click(generate_story_and_choices,[text],[output,choice_1,choice_2,choice_3])
    button_1.click(select_choice,[choice_1],[output,choice_1,choice_2,choice_3])
    button_2.click(select_choice,[choice_2],[output,choice_1,choice_2,choice_3])
    button_3.click(select_choice,[choice_3],[output,choice_1,choice_2,choice_3])
        
        
blocks.queue().launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Pada zaman dahulu, hiduplah seorang gadis
Pada zaman dahulu, hiduplah seorang gadis
